<h1 align=center><font size = 5>Scrapping Data</font></h1>

First we are going to import all the libraries we are going to use

Then we will scrap wikipedia page in oarder to obtain the data required related to postal codes of Canada

Finally, we will save the data (table content) in a .csv file

In [1]:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd

source = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(source,'html.parser')

csv_file = open('wiki_scraper.csv', 'w')
csv_writer = csv.writer(csv_file)
csv_writer.writerow(['PostalCode','Borough','Neighborhood'])

table = soup.find('table', class_='wikitable')

for line in table.find_all('tr'):
    col = line.find_all('td')
    if col:
        postal_code = col[0].text.replace('\n', '')
        borough = col[1].text.replace('\n', '')
        neighborhood = col[2].text.replace('\n', '')
        csv_writer.writerow([postal_code, borough, neighborhood])
        

csv_file.close()

Now, we are going to read the .csv file to create a dataframe

In [2]:
csv_file = open('wiki_scraper.csv', 'r')
df = pd.read_csv(csv_file)
csv_file.close()
df.head()

,PostalCode,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


We are going to process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

In [3]:
df = df[df['Borough'] != 'Not assigned']

If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [4]:
df.loc[df.Neighborhood == 'Not assigned', 'Neighborhood'] = df.Borough
df.loc[df['Neighborhood'].isna(), 'Neighborhood'] = df.Borough
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront
5,M6A,North York,Lawrence Manor / Lawrence Heights
6,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government


More than one neighborhood can exist in one postal code area. Those rows will be combined into one row with the neighborhoods separated with a comma.

In [5]:
df['Neighborhood'] = df['Neighborhood'].apply(lambda x: x.replace(' /',','))
df.head()

,PostalCode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Save the dataframe in a csv file

In [6]:
df.to_csv(r'wiki_df.csv', index = False, header=True)

In [7]:
df.shape

(103, 3)